In [1]:
# Import packages
import numpy as np
from numpy import transpose as T
from numpy import genfromtxt as gftxt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob
import pandas as pd
import os
import cartopy
import cartopy.crs as ccrs
import scipy
from scipy import stats

# Functions 

def includingambiguous_power_law_floes(x):
    return 52.1730619*x**0.137740749
def includingambiguous_power_law_leads(x):
    return 1.334351523*x**0.758163073

def producing_SIC_arrays(source_directory,year,month,sic_output_folder):
    
    count_min_floes = np.load(source_directory+str(year)+str(month)+'_CountFloes.npy')
    count_min_leads = np.load(source_directory+str(year)+str(month)+'_CountLeads.npy')
    count_min_ocean = np.load(source_directory+str(year)+str(month)+'_CountOcean.npy')
    
    # Original
    
    total_values_count = (count_min_floes + count_min_leads + count_min_ocean)
    
    sic = (count_min_floes / (count_min_floes + count_min_leads + count_min_ocean)) *100 
    sic_lead_mask = np.where(total_values_count==count_min_leads,np.nan,sic)
    
    np.save(sic_output_folder+'Original/'+str(year)+str(month)+'_Original_CS2_SIC.npy',sic_lead_mask)
    
    # Corrected 

    corrected_leads_ambiguous = includingambiguous_power_law_leads(count_min_leads)
    corrected_floes_ambiguous = includingambiguous_power_law_floes(count_min_floes)

    total_values_count_corrected = (corrected_floes_ambiguous + corrected_leads_ambiguous + count_min_ocean)


    sic_corrected = (corrected_floes_ambiguous / (corrected_leads_ambiguous + corrected_floes_ambiguous + count_min_ocean))*100
    sic_corrected_lead_mask = np.where(total_values_count_corrected==corrected_leads_ambiguous,np.nan,sic_corrected)
    
    np.save(sic_output_folder+'Corrected/'+str(year)+str(month)+'_Corrected_CS2_SIC.npy',sic_corrected_lead_mask)
    
# Generate Map Area

map_projection = ccrs.LambertAzimuthalEqualArea(central_longitude=-105,central_latitude=74)

# Bin Edges for GRIDDING data (e.g., CS2)
fig1 = plt.figure(figsize=(15,20))

ax1 = plt.axes(projection=map_projection) # This sets the projection of the data
ax1.set_extent([-145, -55, 66, 79], crs=ccrs.PlateCarree())
ax1.coastlines(linewidth=0.2)

x_min, x_max, y_min, y_max = ax1.get_extent()

print("X-axis limits:", x_min, x_max)
print("Y-axis limits:", y_min, y_max)

grid_spacing = 25000
bin_edges=np.arange(x_min,x_max+grid_spacing,grid_spacing)
new_x_edge, new_y_edge = np.meshgrid(bin_edges, bin_edges)

ax1.scatter(new_x_edge,new_y_edge,0.05,color='red',transform=map_projection) 
print('Edges Grid Shape', np.shape(bin_edges))
plt.show()

# Calculate & Save Arrays

source_directory = ('/home/amyswiggs/Chapter2/NWP_CS2_Grids_Cartopy/')

output_folder = ('/home/amyswiggs/Chapter2/NWP_CS2_SIC_Cartopy/')

years = ['2023']
months = ['01','02','03','04','05','09','10','11','12']

for year in years:
    for month in months:
        producing_SIC_arrays(source_directory,year,month,output_folder)
